In [1]:
import os
import numpy as np
import pandas as pd
import keras
import shutil

import matplotlib.pyplot as plt

from keras.layers import Input, Dense, Reshape, Flatten, Dropout, BatchNormalization
from keras.layers import Activation, ZeroPadding2D, Add, Concatenate

from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model, load_model
from keras.optimizers import Adam
import keras.backend as K

Using TensorFlow backend.


In [2]:
image_path = 'celeba-dataset/img_align_celeba'
attributes_path = 'celeba-dataset/list_attr_celeba.csv'
image_folder = 'categories'

### Load attribute data:

In [3]:
attributes_df = pd.read_csv(attributes_path)
attributes_df.columns.values

array(['image_id', '5_o_Clock_Shadow', 'Arched_Eyebrows', 'Attractive',
       'Bags_Under_Eyes', 'Bald', 'Bangs', 'Big_Lips', 'Big_Nose',
       'Black_Hair', 'Blond_Hair', 'Blurry', 'Brown_Hair',
       'Bushy_Eyebrows', 'Chubby', 'Double_Chin', 'Eyeglasses', 'Goatee',
       'Gray_Hair', 'Heavy_Makeup', 'High_Cheekbones', 'Male',
       'Mouth_Slightly_Open', 'Mustache', 'Narrow_Eyes', 'No_Beard',
       'Oval_Face', 'Pale_Skin', 'Pointy_Nose', 'Receding_Hairline',
       'Rosy_Cheeks', 'Sideburns', 'Smiling', 'Straight_Hair',
       'Wavy_Hair', 'Wearing_Earrings', 'Wearing_Hat', 'Wearing_Lipstick',
       'Wearing_Necklace', 'Wearing_Necktie', 'Young'], dtype=object)

In [4]:
attributes_to_keep = ['Black_Hair', 'Blond_Hair', 'Brown_Hair', 'Gray_Hair', 'Male']
attribute_data = attributes_df[attributes_to_keep]
attribute_data = attribute_data.values # note that index starts with 0 and image id with 1!


## Define networks:


### Parameters

In [5]:
noise_size = 100
epochs = 75
save_interval = 1
batch_size = 512
#batches_per_epoch = train_generator.n // batch_size
train_generator= None
data_generator = None
#optimizer = Adam(0.0002, 0.5)
optimizer = Adam(0.00002, 0.5)
batches_per_epoch = 40

In [6]:
optimizer = Adam(0.000002, 0.5)
def wasserstein_loss(y_true, y_pred):
    return K.mean(y_true*y_pred)

### Generate models!

In [7]:
noise_size = 100
nbr_of_features = 5
in_features = Input(shape=[nbr_of_features])
noise_input = Input(shape=[noise_size])


def build_generator():
    generator_input = Concatenate()([noise_input, in_features])
    #generator_input = Input(shape=[latent_dim + nbr_input_features])
    fc = Dense(128*8*8)(generator_input)
    #fc = Dense(16*16*3, activation='sigmoid')(generator_input)
    fc = LeakyReLU(alpha=0.2)(fc)
    image = Reshape((8,8,128))(fc)
    
    image = UpSampling2D()(image) # 16
    image = Conv2D(128, kernel_size=3, padding="same")(image)
    image = BatchNormalization()(image)
    image = LeakyReLU(alpha=0.2)(image)
    
    image = UpSampling2D()(image) # 32
    image = Conv2D(64, kernel_size=3, padding="same")(image)
    image = BatchNormalization()(image)
    image = LeakyReLU(alpha=0.2)(image)
    
    image = UpSampling2D()(image) # 64
    image = Conv2D(32, kernel_size=3, padding="same")(image)
    image = BatchNormalization()(image)
    image = LeakyReLU(alpha=0.2)(image)
    
    #image = UpSampling2D()(image) # 128
    image = Conv2D(3, kernel_size=3, padding="same", activation='sigmoid')(image)
    
    return Model([noise_input, in_features], image)

def build_discriminator():
    
    image_input = Input(shape=(64,64,3))
    
    image = Conv2D(32, kernel_size=5, padding="same", strides=2)(image_input) # 32
    image = LeakyReLU(alpha=0.2)(image)
    
    image = Dropout(0.25)(image)
    image = Conv2D(64, kernel_size=5, padding="same", strides=2)(image) # 16
    image = BatchNormalization()(image)
    image = LeakyReLU(alpha=0.2)(image)
    
    image = Dropout(0.25)(image)
    image = Conv2D(128, kernel_size=5, padding="same", strides=2)(image)# 8
    image = BatchNormalization()(image)
    image = LeakyReLU(alpha=0.2)(image)
    
    image = Dropout(0.25)(image)
    image = Conv2D(256, kernel_size=5, padding="same", strides=1)(image) # 8
    image = BatchNormalization()(image)
    image = LeakyReLU(alpha=0.2)(image)
    
    image_flat = Flatten()(image)
    
    classifier_input = Concatenate()([image_flat, in_features])
    
    fc_size = 64
    hidden = Dense(fc_size)(classifier_input)
    hidden = LeakyReLU(alpha=0.2)(hidden)
    out = Dense(1, activation='sigmoid')(hidden)
    
    return Model([image_input, in_features], out)
#discriminator = Model([in_image, in_features], discriminator_classifier([current_discriminator_cnn(in_image), in_features]))


In [13]:
def plot_history(d_loss_hist, g_loss_hist):
    
    plt.figure()
    plt.plot(d_loss_hist, label='Discriminator')
    plt.plot(g_loss_hist, label='Generator')
    plt.xlabel('Epochs')
    plt.legend(loc='best')
    plt.title('Loss over epochs')
    
feature_matrix = attribute_data[0:25,:]

def numpy_rescaled(x):
    out = x.copy()
    out[x<0] = 0
    out[x>1] = 1
    return out

def save_imgs(epoch, generator):
    r, c = 5, 5
    #noise = np.random.normal(0, 1, (r * c, noise_size))
    #gen_imgs = generator.predict([noise, feature_matrix])
    # 1: female, black hair
    # 2: female, blonde hair
    # 3: female, brown hair
    # 4: male, grey hair
    # 5: male, black hair
    features = np.array([[1, -1, -1, -1, -1], [-1, 1, -1, -1, -1],[-1, -1, 1, -1, -1],[-1, -1, -1, 1, 1], [1, -1, -1, -1, 1]]);
    # Rescale images 0 - 1
    #gen_imgs = 0.5 * gen_imgs + 0.5

    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        
        for j in range(c):
            noise = np.random.normal(0,1, (1, noise_size))
            gen_img = generator.predict([noise, features[i].reshape(1,5)])
            axs[i,j].imshow(gen_img[0,:,:,:])
            axs[i,j].axis('off')
            cnt += 1
    fig.savefig("Images_wasserstein/final_no_wasserstein_%d.png" % epoch)
    plt.close()

def train_gan(discriminator, generator, combined_model, epochs=10, batches_per_epoch=None):
    if batches_per_epoch is None:
        batches_per_epoch = train_generator.n // batch_size


    d_loss_hist = list()
    g_loss_hist = list()
    for epoch in range(epochs):
       
        for i_batch in range(batches_per_epoch):
        # Select a random batch of images
            imgs, y = train_generator.next()
            # one hot to feature
            y = np.argmax(y, axis = 1)
            input_features = attribute_data[y, :]
            
            b_size = imgs.shape[0]
            valid = np.ones((b_size, 1))
            fake = np.zeros((b_size, 1))

            if i_batch%40 == 0:
                print('Currently on batch: %d for epoch: %d' % (i_batch, epoch))

            # Sample noise and generate a batch of new images
            noise = np.random.normal(0, 1, (b_size, noise_size))
            # Generator features
            
            gen_imgs = generator.predict([noise, input_features])
            
            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Train the discriminator (real classified as ones and generated as zeros)
            d_loss_real = discriminator.train_on_batch([imgs, input_features], valid)
            d_loss_fake = discriminator.train_on_batch([gen_imgs, input_features], fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Train Generator
            # ---------------------

            # Train the generator (wants discriminator to mistake images as real)
            g_loss = combined_model.train_on_batch([noise, input_features], valid)

        d_loss_hist.append(d_loss[0])
        g_loss_hist.append(g_loss)
        # Plot the progress
        print ("epoch: %d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

        # If at save interval => save generated image samples
        if epoch % save_interval == 0:
            save_imgs(epoch, generator)
        if epoch % 10 == 0:
            generator.save('generator_fine_tune_2_epoch_%d.h5' % epoch)
            discriminator.save('discriminator_fine_tune_2_epoch_%d.h5' % epoch)
    
    return d_loss_hist, g_loss_hist

## Step 1

In [9]:
step = 1
# First generator step
generator = build_generator()
#current_generator_model.summary()

# First Discriminator step, only classifier
discriminator = build_discriminator()
#discriminator_classifier.summary()



In [17]:
generator = load_model('generator_epoch_50.h5')
discriminator = load_model('discriminator_epoch_50.h5')

#save_imgs(-1, generator)

/home/student/anaconda3/envs/dml_gpu/lib/python3.6/site-packages/keras/engine/saving.py:269: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [14]:
save_imgs(-1, generator)


Train step 1

In [17]:
#target_size = (64, 64)
#input_shape = (target_size[0], target_size[1], 3)

def get_data_generators(image_size):
    data_generator = keras.preprocessing.image.ImageDataGenerator(rescale = 1/255)

    train_generator = data_generator.flow_from_directory(image_folder, 
                                                         target_size=(image_size, image_size), 
                                                         batch_size=batch_size)
    return data_generator, train_generator


In [22]:
discriminator.compile(loss='binary_crossentropy', 
                      optimizer=optimizer,
                      metrics=['accuracy'])

# The generator takes noise as input and generates imgs
img = generator([noise_input, in_features])

# For the combined model we will only train the generator
discriminator.trainable = False

# The discriminator takes generated images as input and determines validity
valid = discriminator([img, in_features])

# The combined model  (stacked generator and discriminator)
# Trains the generator to fool the discriminator
combined = Model([noise_input, in_features], valid)
combined.compile(loss='binary_crossentropy', optimizer=optimizer)

In [12]:
data_generator, train_generator = get_data_generators(64)

Found 202599 images belonging to 202599 classes.


In [23]:
d_loss_hist, g_loss_hist = train_gan(discriminator, generator, combined, epochs=epochs, batches_per_epoch=batches_per_epoch)
plot_history(d_loss_hist, g_loss_hist)

Currently on batch: 0 for epoch: 0
epoch: 0 [D loss: 0.408586, acc.: 81.64%] [G loss: 0.015929]
Currently on batch: 0 for epoch: 1
epoch: 1 [D loss: 1.122392, acc.: 58.11%] [G loss: 0.008966]
Currently on batch: 0 for epoch: 2
epoch: 2 [D loss: 2.006832, acc.: 50.29%] [G loss: 0.007024]
Currently on batch: 0 for epoch: 3
epoch: 3 [D loss: 2.529087, acc.: 49.32%] [G loss: 0.006569]
Currently on batch: 0 for epoch: 4
epoch: 4 [D loss: 2.836002, acc.: 50.10%] [G loss: 0.003931]
Currently on batch: 0 for epoch: 5
epoch: 5 [D loss: 3.350276, acc.: 49.51%] [G loss: 0.002307]
Currently on batch: 0 for epoch: 6
epoch: 6 [D loss: 3.327423, acc.: 49.22%] [G loss: 0.002714]
Currently on batch: 0 for epoch: 7
epoch: 7 [D loss: 3.623621, acc.: 49.80%] [G loss: 0.002403]
Currently on batch: 0 for epoch: 8
epoch: 8 [D loss: 3.703610, acc.: 49.61%] [G loss: 0.002207]
Currently on batch: 0 for epoch: 9
epoch: 9 [D loss: 3.754653, acc.: 49.71%] [G loss: 0.001253]
Currently on batch: 0 for epoch: 10
epoc

KeyboardInterrupt: 

Save the models from step 1

In [24]:
generator.save('generator_finalefinetune2.h5')
discriminator.save('discriminator_finalefinetune2.h5')